In [1]:
import os
from dotenv import load_dotenv

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.ingestion import IngestionPipeline
import Stemmer
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

load_dotenv()

resource module not available on Windows


True

Azure OpenAI LLM and Embedding connection:

In [8]:
llm = AzureOpenAI(
    engine=os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT"),
    model=os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_LLM_API_VERSION"),
)


embed_model = AzureOpenAIEmbedding(
    model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    deployment_name=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION"),
)


Settings.llm = llm
Settings.embed_model = embed_model

Ollama for open source LLM and Embedding models:

In [ ]:
# ollama_llm = Ollama(model="deepseek-r1:7b", request_timeout=600.0)

# ollama_embed_model = OllamaEmbedding(
#     model_name="nomic-embed-text:latest",
#     base_url="http://localhost:11434"
# )

# Settings.llm = ollama_llm
# Settings.embed_model = ollama_embed_model

In [ ]:
# pass_embedding = embed_model.get_text_embedding_batch(
#     ["This is a passage!", "This is another passage"], show_progress=True
# )
# print(pass_embedding)
# print(len(pass_embedding[0]))

Load pdf file/s from data folder:

In [2]:
profiles_folder = "../data"
documents = SimpleDirectoryReader(profiles_folder).load_data()
print(len(documents))

22


Split text by sentence

In [3]:
# initialize node parser
splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)

nodes = splitter.get_nodes_from_documents(documents)
print(len(nodes))
nodes

52


[TextNode(id_='922c0d5e-47e2-4c6c-98c7-3ffba9d9cab7', embedding=None, metadata={'page_label': '1', 'file_name': 'deepseek-r1-paper.pdf', 'file_path': 'c:\\Users\\nazaizah\\OneDrive\\PycharmProjects\\rag-tutorial\\data\\deepseek-r1-paper.pdf', 'file_type': 'application/pdf', 'file_size': 1312189, 'creation_date': '2025-02-02', 'last_modified_date': '2025-02-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='660d5ed4-4949-419a-899d-77648500acdd', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'deepseek-r1-paper.pdf', 'file_path': 'c:\\Users\\nazaizah\\OneDrive\\PycharmProjects\\rag-tutorial\\data\\deepseek-r1-paper.pdf', 'file_type': 'application/pdf', 'file_size': 1312189,

In [ ]:
nodes[0].metadata

Split with more metadata:

In [ ]:
transformations = [
    SentenceSplitter(chunk_size=500, chunk_overlap=50),
    TitleExtractor(nodes=3),
    # QuestionsAnsweredExtractor(questions=3),
    # SummaryExtractor(summaries=["prev", "self"]),
    # KeywordExtractor(keywords=10),
    # EntityExtractor(prediction_threshold=0.5),
]
pipeline = IngestionPipeline(transformations=transformations)

advanced_nodes = pipeline.run(documents=documents)
print(len(advanced_nodes))
advanced_nodes

In [ ]:
advanced_nodes[0].metadata

In [ ]:
advanced_nodes[-1].metadata

Question which we will use for the retrieve and query:

In [4]:
question = "which reinforcement learning algorithm used in the deepseek r1 training?"
# question = "can you explain how deepseek r1 model was trained?"
# question = "can you explain more about the cold start?"
# question = "list all the unsuccessful attempts mentioned in the paper?"
question

'which reinforcement learning algorithm used in the deepseek r1 training?'

Key words search using BM25:

In [9]:
# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,  # advanced_nodes
    similarity_top_k=10,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)
bm25_retriever

In [ ]:
# will retrieve context from specific companies
bm25_retrieved_nodes = bm25_retriever.retrieve(question)
for node in bm25_retrieved_nodes:
    display_source_node(node, source_length=5000)

In [ ]:
bm25_query_engine = RetrieverQueryEngine.from_args(bm25_retriever)

response = bm25_query_engine.query(question)
display_response(
    response, source_length=5000, show_source=True, show_source_metadata=True
)

Vector Search:

In [10]:
vector_store = LanceDBVectorStore(
    uri="./lancedb", mode="overwrite", query_type="vector"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

vector_index = VectorStoreIndex(
    nodes=nodes, show_progress=True, storage_context=storage_context
)
# advanced_nodes
# configure retriever
vector_retriever = VectorIndexRetriever(
    index=vector_index,
    similarity_top_k=10,
)

Generating embeddings:   0%|          | 0/52 [00:00<?, ?it/s]

In [ ]:
vector_retrieved_nodes = vector_retriever.retrieve(question)
for node in vector_retrieved_nodes:
    display_source_node(node, source_length=5000)

In [13]:
vector_query_engine = RetrieverQueryEngine.from_args(vector_retriever)

response = vector_query_engine.query(question)
display_response(
    response, source_length=5000, show_source=True, show_source_metadata=True
)

**`Final Response:`** The specific reinforcement learning algorithm used in the training of DeepSeek-R1 is not explicitly mentioned. However, the training involves a large-scale reinforcement learning process that focuses on developing reasoning capabilities without relying on supervised fine-tuning as a preliminary step.

---

**`Source Node 1/10`**

**Node ID:** 922c0d5e-47e2-4c6c-98c7-3ffba9d9cab7<br>**Similarity:** 0.7797601819038391<br>**Text:** DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
(1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama.<br>**Metadata:** {'page_label': '1', 'file_name': 'deepseek-r1-paper.pdf', 'file_path': 'c:\\Users\\nazaizah\\OneDrive\\PycharmProjects\\rag-tutorial\\data\\deepseek-r1-paper.pdf', 'file_type': 'application/pdf', 'file_size': 1312189, 'creation_date': '2025-02-02', 'last_modified_date': '2025-02-02'}<br>

---

...

**`Source Node 9/10`**

**Node ID:** 0c3dc7ce-423c-4575-a667-265bccde7640<br>**Similarity:** 0.7488131523132324<br>**Text:** As depicted in Table 1, this template
requires DeepSeek-R1-Zero to first produce a reasoning process, followed by the final answer.
We intentionally limit our constraints to this structural format, avoiding any content-specific
biases—such as mandating reflective reasoning or promoting particular problem-solving strate-
gies—to ensure that we can accurately observe the model’s natural progression during the RL
process.
2.2.4. Performance, Self-evolution Process and Aha Moment of DeepSeek-R1-Zero
Performance of DeepSeek-R1-Zero Figure 2 depicts the performance trajectory of DeepSeek-
R1-Zero on the AIME 2024 benchmark throughout the RL training process. As illustrated,
DeepSeek-R1-Zero demonstrates a steady and consistent enhancement in performance as the
RL training advances. Notably, the average pass@1 score on AIME 2024 shows a significant
increase, jumping from an initial 15.6% to an impressive 71.0%, reaching performance levels
comparable to OpenAI-o1-0912. This significant improvement highlights the efficacy of our RL
algorithm in optimizing the model’s performance over time.
Table 2 provides a comparative analysis between DeepSeek-R1-Zero and OpenAI’s o1-0912
models across a variety of reasoning-related benchmarks. The findings reveal that RL empowers
6<br>**Metadata:** {'page_label': '6', 'file_name': 'deepseek-r1-paper.pdf', 'file_path': 'c:\\Users\\nazaizah\\OneDrive\\PycharmProjects\\rag-tutorial\\data\\deepseek-r1-paper.pdf', 'file_type': 'application/pdf', 'file_size': 1312189, 'creation_date': '2025-02-02', 'last_modified_date': '2025-02-02'}<br>

---

**`Source Node 10/10`**

**Node ID:** 93bddeb7-c8c6-4c7b-8064-db6725ec69a3<br>**Similarity:** 0.7461886405944824<br>**Text:** Figure 3 |The average response length of DeepSeek-R1-Zero on the training set during the RL
process. DeepSeek-R1-Zero naturally learns to solve reasoning tasks with more thinking time.
ment throughout the training process. This improvement is not the result of external adjustments
but rather an intrinsic development within the model. DeepSeek-R1-Zero naturally acquires the
ability to solve increasingly complex reasoning tasks by leveraging extended test-time compu-
tation. This computation ranges from generating hundreds to thousands of reasoning tokens,
allowing the model to explore and refine its thought processes in greater depth.
One of the most remarkable aspects of this self-evolution is the emergence of sophisticated
behaviors as the test-time computation increases. Behaviors such as reflection—where the model
revisits and reevaluates its previous steps—and the exploration of alternative approaches to
problem-solving arise spontaneously. These behaviors are not explicitly programmed but instead
emerge as a result of the model’s interaction with the reinforcement learning environment. This
spontaneous development significantly enhances DeepSeek-R1-Zero’s reasoning capabilities,
enabling it to tackle more challenging tasks with greater efficiency and accuracy.
Aha Moment of DeepSeek-R1-ZeroA particularly intriguing phenomenon observed during
the training of DeepSeek-R1-Zero is the occurrence of an “aha moment”. This moment, as
illustrated in Table 3, occurs in an intermediate version of the model. During this phase,
DeepSeek-R1-Zero learns to allocate more thinking time to a problem by reevaluating its initial
approach. This behavior is not only a testament to the model’s growing reasoning abilities
but also a captivating example of how reinforcement learning can lead to unexpected and
sophisticated outcomes.
This moment is not only an “aha moment” for the model but also for the researchers
observing its behavior. It underscores the power and beauty of reinforcement learning: rather
than explicitly teaching the model on how to solve a problem, we simply provide it with the
right incentives, and it autonomously develops advanced problem-solving strategies.<br>**Metadata:** {'page_label': '8', 'file_name': 'deepseek-r1-paper.pdf', 'file_path': 'c:\\Users\\nazaizah\\OneDrive\\PycharmProjects\\rag-tutorial\\data\\deepseek-r1-paper.pdf', 'file_type': 'application/pdf', 'file_size': 1312189, 'creation_date': '2025-02-02', 'last_modified_date': '2025-02-02'}<br>

Hybrid Search:

In [22]:
hybrid_retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=10,
    num_queries=1,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=False,
    verbose=True,
)

In [ ]:
hybrid_retrieved_nodes = hybrid_retriever.retrieve(question)
for node in hybrid_retrieved_nodes:
    display_source_node(node, source_length=5000)

In [27]:
question = "can you list all the advantages of the deepseek r1 model over over models?"

question = "can explain in detail about reward modeling in which part of the paper it was mentioned?"

In [ ]:
hybrid_query_engine = RetrieverQueryEngine.from_args(hybrid_retriever)

response = hybrid_query_engine.query(question)

display_response(
    response, source_length=5000, show_source=True, show_source_metadata=True
)

View Prompt:

In [ ]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))


prompts_dict = hybrid_query_engine.get_prompts()
display_prompt_dict(prompts_dict)

Links:

https://arxiv.org/pdf/2501.12948

https://docs.llamaindex.ai/en/v0.10.33/examples/vector_stores/SimpleIndexDemo/

https://docs.llamaindex.ai/en/stable/examples/retrievers/bm25_retriever/

https://docs.llamaindex.ai/en/stable/examples/retrievers/reciprocal_rerank_fusion/


https://docs.llamaindex.ai/en/stable/module_guides/indexing/metadata_extraction/

Advanced Links:

https://docs.llamaindex.ai/en/stable/examples/retrievers/auto_merging_retriever/

https://docs.llamaindex.ai/en/stable/examples/workflow/rag/

Ollama:

https://ollama.com/